# Решающие деревья

###  Загрузка данных

Для улучшения качества классификации, устраним пропуски и прошкалируем данные с использованием тестового набора.

In [164]:
train = pd.read_csv('titanic.train.csv')    
test = pd.read_csv('titanic.test.csv')
targets = train.Survived
train.drop('Survived', 1, inplace=True)

combined = train.append(test)
combined.reset_index(inplace=True)
combined.drop('index',inplace=True,axis=1)

In [165]:
combined.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Извлечение имен пассажиров

Рассматривая имена пассажиров, можно было бы задаться вопросом, как их обработать, чтобы извлечь легко интерпретируемую информацию.

Посмотрим более внимательно на примеры:

- Braund, <b> Mr.</b> Owen Harris	
- Heikkinen, <b>Miss.</b> Laina
- Oliva y Ocana, <b>Dona.</b> Fermina
- Peter, <b>Master.</b> Michael J

Можно заметить, что у каждого имени есть приставка! Это может быть простая мисс или миссис, но иногда это может быть что-то более сложное, как Мастер, Сэр или Дона. В этом случае мы могли бы ввести дополнительную информацию о социальном статусе, просто проанализировав имя и извлекая заголовок.

In [166]:
combined['Title'] = combined['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())

titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}

combined['Title'] = combined.Title.map(titles)

Эта функция анализирует имена и извлекает заголовки. Затем он отображает заголовки в соответствующей категории.
Мы выбрали:

- Officer
- Royalty 
- Mr
- Mrs
- Miss
- Master

In [167]:
combined.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


### Обработка возраста

Изучая данные, можно увидеть, что в переменной Age отсутствует 177 значений. Это большое количество (~ 13% от набора данных). Просто заменить их средним или средним возрастом может быть не лучшим решением, так как возраст может отличаться по группам и категориям пассажиров.

Чтобы понять, почему, давайте сгруппируем наш набор данных по полу, типу и классу пассажиров.

In [168]:
grouped = combined.groupby(['Sex','Pclass','Title'])
grouped.median()

PassengerId   Age  SibSp  Parch      Fare
Sex    Pclass Title                                             
female 1      Miss           529.5  30.0    0.0    0.0   99.9625
              Mrs            853.5  45.0    1.0    0.0   78.1125
              Officer        797.0  49.0    0.0    0.0   25.9292
              Royalty        760.0  39.0    0.0    0.0   86.5000
       2      Miss           606.5  20.0    0.0    0.0   20.2500
              Mrs            533.0  30.0    1.0    0.0   26.0000
       3      Miss           603.5  18.0    0.0    0.0    8.0500
              Mrs            668.5  31.0    1.0    1.0   15.5000
male   1      Master         803.0   6.0    1.0    2.0  134.5000
              Mr             634.0  41.5    0.0    0.0   47.1000
              Officer        678.0  52.0    0.0    0.0   37.5500
              Royalty        600.0  40.0    0.0    0.0   27.7208
       2      Master         550.0   2.0    1.0    1.0   26.0000
              Mr             723.5  30.0    0.0    0.0   13.0000
              Officer        513.0  41.5    0.0    0.0   13.0000
       3      Master         789.0   6.0    3.0    1.0   22.3583
              Mr             640.5  26.0    0.0    0.0    7.8958

Посмотрите на столбец среднего возраста, как это значение может отличаться в зависимости от Sex, Pclass и Title?

Например:

- Если пассажир женского пола, из класса Pclass 1 - средний возраст 39 лет.
- Если пассажир мужского пола, из класса Pclass 3, с титулом мистера - средний возраст 26 лет.

Давайте создадим функцию, которая заполняет отсутствующий возраст.

In [169]:
def fillAges(row):
    if row['Sex']=='female' and row['Pclass'] == 1:
        if row['Title'] == 'Miss':
            return 30
        elif row['Title'] == 'Mrs':
            return 45
        elif row['Title'] == 'Officer':
            return 49
        elif row['Title'] == 'Royalty':
            return 39

    elif row['Sex']=='female' and row['Pclass'] == 2:
        if row['Title'] == 'Miss':
            return 20
        elif row['Title'] == 'Mrs':
            return 30

    elif row['Sex']=='female' and row['Pclass'] == 3:
        if row['Title'] == 'Miss':
            return 18
        elif row['Title'] == 'Mrs':
            return 31

    elif row['Sex']=='male' and row['Pclass'] == 1:
        if row['Title'] == 'Master':
            return 6
        elif row['Title'] == 'Mr':
            return 41.5
        elif row['Title'] == 'Officer':
            return 52
        elif row['Title'] == 'Royalty':
            return 40

    elif row['Sex']=='male' and row['Pclass'] == 2:
        if row['Title'] == 'Master':
            return 2
        elif row['Title'] == 'Mr':
            return 30
        elif row['Title'] == 'Officer':
            return 41.5

    elif row['Sex']=='male' and row['Pclass'] == 3:
        if row['Title'] == 'Master':
            return 6
        elif row['Title'] == 'Mr':
            return 26

combined.Age = combined.apply(lambda r : fillAges(r) if np.isnan(r['Age']) else r['Age'], axis=1)

In [170]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1309 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Title          1309 non-null object
dtypes: float64(2), int64(4), object(6)
memory usage: 122.8+ KB


Отлично. Недостающие возрасты были восстановлены.

Тем не менее, мы замечаем отсутствие значения в Fare, два отсутствующих значения в Embarked и много недостающих значений в Cabin. Мы вернемся к этим переменным позже.

Давайте теперь обработаем имена.

In [171]:
combined.drop('Name',axis=1,inplace=True)

titles_dummies = pd.get_dummies(combined['Title'],prefix='Title')
combined = pd.concat([combined,titles_dummies],axis=1)

combined.drop('Title',axis=1,inplace=True)

In [172]:
combined.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
0,1,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,0,1,0,0,0
1,2,1,female,38.0,1,0,PC 17599,71.2833,C85,C,0,0,0,1,0,0
2,3,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1,0,0,0,0
3,4,1,female,35.0,1,0,113803,53.1000,C123,S,0,0,0,1,0,0
4,5,3,male,35.0,0,0,373450,8.0500,NaN,S,0,0,1,0,0,0


В итоге :
- больше нет имен.
- появились новые двоичные переменные (Title_X).
     - Например, если Title_Mr = 1, то соответствующий заголовок - Mr.

### Обработка остальной части данных

In [173]:
combined.Fare.fillna(combined.Fare.mean(),inplace=True)

In [174]:
combined.Embarked.fillna('S',inplace=True)
embarked_dummies = pd.get_dummies(combined['Embarked'],prefix='Embarked')
combined = pd.concat([combined,embarked_dummies],axis=1)
combined.drop('Embarked',axis=1,inplace=True)

In [175]:
combined.Cabin.fillna('U',inplace=True)
combined['Cabin'] = combined['Cabin'].map(lambda c : c[0]) 
cabin_dummies = pd.get_dummies(combined['Cabin'],prefix='Cabin')
combined = pd.concat([combined,cabin_dummies],axis=1)
combined.drop('Cabin',axis=1,inplace=True)

In [176]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 26 columns):
PassengerId      1309 non-null int64
Pclass           1309 non-null int64
Sex              1309 non-null object
Age              1309 non-null float64
SibSp            1309 non-null int64
Parch            1309 non-null int64
Ticket           1309 non-null object
Fare             1309 non-null float64
Title_Master     1309 non-null uint8
Title_Miss       1309 non-null uint8
Title_Mr         1309 non-null uint8
Title_Mrs        1309 non-null uint8
Title_Officer    1309 non-null uint8
Title_Royalty    1309 non-null uint8
Embarked_C       1309 non-null uint8
Embarked_Q       1309 non-null uint8
Embarked_S       1309 non-null uint8
Cabin_A          1309 non-null uint8
Cabin_B          1309 non-null uint8
Cabin_C          1309 non-null uint8
Cabin_D          1309 non-null uint8
Cabin_E          1309 non-null uint8
Cabin_F          1309 non-null uint8
Cabin_G          1309 non-null uint8

Отлично, больше нет пропущенных значений.

In [177]:
combined.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Title_Master,Title_Miss,...,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U
0,1,3,male,22.0,1,0,A/5 21171,7.2500,0,0,...,1,0,0,0,0,0,0,0,0,1
1,2,1,female,38.0,1,0,PC 17599,71.2833,0,0,...,0,0,0,1,0,0,0,0,0,0
2,3,3,female,26.0,0,0,STON/O2. 3101282,7.9250,0,1,...,1,0,0,0,0,0,0,0,0,1
3,4,1,female,35.0,1,0,113803,53.1000,0,0,...,1,0,0,1,0,0,0,0,0,0
4,5,3,male,35.0,0,0,373450,8.0500,0,0,...,1,0,0,0,0,0,0,0,0,1


In [178]:
combined['Sex'] = combined['Sex'].map({'male':1,'female':0})

In [179]:
pclass_dummies = pd.get_dummies(combined['Pclass'],prefix="Pclass")
combined = pd.concat([combined,pclass_dummies],axis=1)    
combined.drop('Pclass',axis=1,inplace=True)

In [180]:
def cleanTicket(ticket):
    ticket = ticket.replace('.','')
    ticket = ticket.replace('/','')
    ticket = ticket.split()
    ticket = map(lambda t : t.strip() , ticket)
    ticket = list(filter(lambda t : not t.isdigit(), ticket))
    if len(ticket) > 0:
        return ticket[0]
    else: 
        return 'XXX'


combined['Ticket'] = combined['Ticket'].map(cleanTicket)
tickets_dummies = pd.get_dummies(combined['Ticket'],prefix='Ticket')
combined = pd.concat([combined, tickets_dummies],axis=1)
combined.drop('Ticket',inplace=True,axis=1)

* Эта функция выполняет предварительную обработку билетов путем извлечения префикса билета. Если извлечение префикса невозможно, возвращается XXX.
* Затем она кодирует префиксы с использованием фиктивной кодировки.

### Обработка семей

Создадим новые переменные на основе размера семьи.

In [181]:
combined['FamilySize'] = combined['Parch'] + combined['SibSp'] + 1
combined['Singleton'] = combined['FamilySize'].map(lambda s : 1 if s == 1 else 0)
combined['SmallFamily'] = combined['FamilySize'].map(lambda s : 1 if 2<=s<=4 else 0)
combined['LargeFamily'] = combined['FamilySize'].map(lambda s : 1 if 5<=s else 0)

Эта функция создает 4 новые переменные:

- FamilySize: общее количество родственников, включая пассажира (его / ее).
- Sigleton: булевская переменная, описывающая семейства размером = 1
- SmallFamily: логическая переменная, описывающая семейства из 2 <= size <= 4
- LargeFamily: логическая переменная, описывающая семейства из 5 < size

In [182]:
combined.head()

,PassengerId,Sex,Age,SibSp,Parch,Fare,Title_Master,Title_Miss,Title_Mr,Title_Mrs,...,Ticket_STONO2,Ticket_STONOQ,Ticket_SWPP,Ticket_WC,Ticket_WEP,Ticket_XXX,FamilySize,Singleton,SmallFamily,LargeFamily
0,1,1,22.0,1,0,7.2500,0,0,1,0,...,0,0,0,0,0,0,2,0,1,0
1,2,0,38.0,1,0,71.2833,0,0,0,1,...,0,0,0,0,0,0,2,0,1,0
2,3,0,26.0,0,0,7.9250,0,1,0,0,...,1,0,0,0,0,0,1,1,0,0
3,4,0,35.0,1,0,53.1000,0,0,0,1,...,0,0,0,0,0,1,2,0,1,0
4,5,1,35.0,0,0,8.0500,0,0,1,0,...,0,0,0,0,0,1,1,1,0,0


Как вы можете видеть, они имеют разные области значения. Давайте нормализуем их все, кроме PassengerId, в единичном интервале.

In [184]:
features = list(combined.columns)
features.remove('PassengerId')
combined[features] = combined[features].apply(lambda x: x/x.max(), axis=0)

Возвращаясь к нашей проблеме, теперь мы должны:

1. Разбить объединенный набор данных в набор для обучения и тестирования.
2. Построить предсказания.
3. Оцените модель.

Чтобы оценить нашу модель, мы будем использовать 5-fold cross validation с метрикой Accuracy.

In [185]:
def compute_score(clf, X, y, scoring='accuracy'):
    xval = cross_val_score(clf, X, y, cv = 5,scoring=scoring)
    return np.mean(xval)

In [208]:
targets = pd.read_csv('titanic.train.csv').Survived

train = combined.ix[0:890]
test = combined.ix[891:]

## Выбор признаков

На данный момент у нас есть 68 признаков. Это довольно большое число.
Предлагается выбрать из них лучшие.

Отбор признаков имеет много преимуществ:

- Это уменьшает избыточность среди данных
- Это ускоряет процесс обучения
- Это уменьшает overfitting

In [209]:
classifier = ExtraTreesClassifier(n_estimators=200)
classifier = clf.fit(train, targets)

In [210]:
features = pd.DataFrame()

features['feature'] = train.columns
features['importance'] = clf.feature_importances_

In [211]:
features.sort_values(['importance'],ascending=False)[:10]

,feature,importance
8,Title_Mr,0.541909
64,FamilySize,0.140798
5,Fare,0.115458
10,Title_Officer,0.047081
0,PassengerId,0.042821
26,Pclass_3,0.037470
2,Age,0.021493
25,Pclass_2,0.021451
60,Ticket_SWPP,0.013883
57,Ticket_STONO,0.010900


Как вы можете заметить, наиболее важными являются Title_Mr, Age, Fare и Sex.
Существует также важная зависимость с Passenger_Id.

Давайте теперь преобразуем наши данные.

In [213]:
model = SelectFromModel(classifier, prefit=True)

train_new = model.transform(train)
test_new = model.transform(test)

### Настройка гиперпараметров

Для нахождения наилучших параметров, будем использовать **GridSearch**.

Преимущество прохода по сетке в том, что его легко имплементировать самому и легко распараллелить. Однако у него есть и серьезные недостатки:

- Он перебирает много заведомо неудачных точек. Допустим, уже имеется набор каких-то конфигураций с результатами или какая-то другая информация. Человек может понять, какие конфигурации точно дадут неприемлимый результат, и догадается не проверять лишний раз эти регионы. Grid search так делать не умеет. 

- Если гиперпараметров много, то размер «ячейки» приходится делать слишком крупным, и можно упустить хороший оптимум. Таким образом, если включить в пространство поиска много лишних гиперпараметров, никак не влияющих на результат, то grid search будет работать намного хуже при том же числе итераций.

In [218]:
ensemble = DecisionTreeClassifier(max_features='sqrt')

parameter_grid = {
    'max_depth' : [2, 3, 4, 5, 6, 7, 8, 9, 10],
    'criterion': ['gini','entropy']
}

grid_search = GridSearchCV(
    ensemble,
    param_grid = parameter_grid,
    cv = StratifiedKFold(n_splits=5).split(train, targets)
)

grid_search.fit(train_new, targets)

print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

Best score: 0.8092031425364759
Best parameters: {'criterion': 'entropy', 'max_depth': 5}
